# Capítulo 9

# Mercado financeiro e eventos especiais

## Consultando o preço de ações

In [ ]:
!pip install yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df_acao = yf.download('TAEE11.SA',end='2022-02-04')

In [ ]:
df_acao.round(2)\
  .head()

In [ ]:
#Dados de treino
df_treino = pd.DataFrame()
df_treino['ds'] = df_acao.index[:3400]
df_treino['y'] = df_acao['Adj Close'][:3400].values
#Dados de teste
df_teste = pd.DataFrame()
df_teste['ds'] = df_acao.index[3400:]
df_teste['y'] = df_acao['Adj Close'][3400:].values

#Previsão com o Prophet
from fbprophet import Prophet
modelo = Prophet(seasonality_mode='additive',daily_seasonality=False,weekly_seasonality=True,yearly_seasonality=True,
                 n_changepoints = 5)
modelo.fit(df_treino)
dataframefuturo = modelo.make_future_dataframe(freq='D',periods=184)
previsao = modelo.predict(dataframefuturo);

#Construção do gráfico
fig = modelo.plot(previsao)
plt.plot(df_teste['ds'],df_teste['y'],'.r')

In [ ]:
df_teste['ds'][125]

In [ ]:
dataframefuturo['ds'][3583]

## Feriados

In [ ]:
modelo = Prophet(seasonality_mode='additive',daily_seasonality=False,weekly_seasonality=True,yearly_seasonality=True,
                 n_changepoints = 5)
#Adicionando feriados do Brasil
modelo.add_country_holidays(country_name='BR')

modelo.fit(df_treino)
dataframefuturo = modelo.make_future_dataframe(freq='D',periods=185)
previsao = modelo.predict(dataframefuturo);

#Construção do gráfico
fig = modelo.plot(previsao)
plt.plot(df_teste['ds'],df_teste['y'],'.r')

In [ ]:
modelo.plot_components(previsao);

In [ ]:
modelo.train_holiday_names

In [ ]:
previsao.columns

In [ ]:
col_feriados = previsao[['ds',*modelo.train_holiday_names.tolist()]]

In [ ]:
col_feriados.columns = ['ds','Ano_novo','Tiradentes','Trabalho','Independencia','Aparecida','Finados','Republica','Natal']

In [ ]:
contrib_feriados = col_feriados.query('Ano_novo !=0 or Tiradentes !=0 or Trabalho != 0 or Independencia != 0 or Aparecida != 0 or Finados != 0 or Republica != 0 or Natal != 0')

In [ ]:
resumo_feriados = pd.DataFrame()
resumo_feriados['ds'] = contrib_feriados['ds']

In [ ]:
resumo_feriados['feriados'] = contrib_feriados[['Ano_novo','Tiradentes','Trabalho','Independencia','Aparecida','Finados','Republica','Natal']].sum(axis=1)

In [ ]:
resumo_feriados.round(2)

## Datas customizadas

In [ ]:
informacoes = yf.Ticker('TAEE11.SA')
for chave, valor in informacoes.info.items():
  print(chave,':',valor)

In [ ]:
dividendos = pd.DataFrame(informacoes.dividends)
dividendos.head()

In [ ]:
dividendos.shape

In [ ]:
datas_dividendos = pd.DataFrame({
    'holiday': 'dividendos',
    'ds': pd.to_datetime(dividendos.index),
    'lower_window': 0,
    'upper_window': 0,
})

In [ ]:
modelo = Prophet(seasonality_mode='additive',daily_seasonality=False,weekly_seasonality=True,yearly_seasonality=True,
                 n_changepoints = 5,holidays=datas_dividendos)

modelo.fit(df_treino)
dataframefuturo = modelo.make_future_dataframe(freq='D',periods=185)
previsao = modelo.predict(dataframefuturo);

In [ ]:
modelo.plot_components(previsao,'holidays');

## Importância individual dos eventos

In [ ]:
from numpy.random import randint
prior_scale = randint(0,10,41)

In [ ]:
prior_scale

In [ ]:
datas_dividendos = pd.DataFrame({
    'holiday': 'dividendos',
    'ds': pd.to_datetime(dividendos.index),
    'lower_window': 0,
    'upper_window': 0,
    'prior_scale': prior_scale
})

In [ ]:
pd.options.display.max_columns = 5
datas_dividendos.head()

## Gráficos de velas

In [ ]:
import plotly.graph_objects as go
fig = go.Figure(data=[go.Candlestick(x=df_acao.index,
                open=df_acao['Open'],
                high=df_acao['High'],
                low=df_acao['Low'],
                close=df_acao['Close'],
                increasing_line_color= 'black', decreasing_line_color= 'gray'
)])
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()